***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 11 13:35 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket_323040972' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
!gsutil ls gs://bucket_323040972

gs://bucket_323040972/graphframes.sh
gs://bucket_323040972/multistream10_preprocessed.parquet
gs://bucket_323040972/multistream11_part2_preprocessed.parquet
gs://bucket_323040972/multistream11_preprocessed.parquet
gs://bucket_323040972/multistream12_part2_preprocessed.parquet
gs://bucket_323040972/multistream12_preprocessed.parquet
gs://bucket_323040972/multistream13_part2_preprocessed.parquet
gs://bucket_323040972/multistream13_preprocessed.parquet
gs://bucket_323040972/multistream14_part2_preprocessed.parquet
gs://bucket_323040972/multistream14_preprocessed.parquet
gs://bucket_323040972/multistream15_part2_preprocessed.parquet
gs://bucket_323040972/multistream15_part3_preprocessed.parquet
gs://bucket_323040972/multistream15_preprocessed.parquet
gs://bucket_323040972/multistream16_part2_preprocessed.parquet
gs://bucket_323040972/multistream16_part3_preprocessed.parquet
gs://bucket_323040972/multistream16_preprocessed.parquet
gs://bucket_323040972/multistream17_part2_preprocessed.parqu

In [9]:
parquet_path = "gs://bucket_323040972/multistream*_preprocessed.parquet"
parquetFile = spark.read.parquet(parquet_path)

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [t for t in tokens if t not in all_stopwords]
  tf_counts = Counter(tokens)

  resualt = []
  for token, count in tf_counts.items():
    resualt.append((token, (id, count)))

  return resualt


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  result = postings.map(lambda x: (x[0], len(x[1])))
  return result


def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # Map each (token, posting_list) to (bucket_id, (token, posting_list))
  bucket_posting = postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))

  # Group all (token, posting_list) pairs by their bucket_id.
  grouped_by_bucket = bucket_posting.groupByKey()

  # For each bucket, write its posting lists to disk and return the posting locations.
  posting_locations_rdd  = grouped_by_bucket.map(
      lambda bckt: InvertedIndex.write_a_posting_list((str(bckt[0]), list(bckt[1])),bucket_name)
  )
  return posting_locations_rdd

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
# test index construction time
assert index_const_time < 60*120
print(index_const_time)

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''

  edges = pages.flatMap(lambda page: [(page[0], anchor[0]) for anchor in page[1]]) \
               .distinct()

  
  src_vertices = edges.map(lambda edge: edge[0])
  dst_vertices = edges.map(lambda edge: edge[1])
  vertices = src_vertices.union(dst_vertices).distinct().map(lambda x: Row(id=x))

  return edges, vertices

In [ ]:
t_start = time()
parquet_path = f"gs://{bucket_name}/multistream*_preprocessed.parquet"
pages_links = spark.read.parquet(parquet_path).select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()
print("End!!!")

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 6.49 
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag

# Build doc_id → title shards and a Title Inverted Index

title dictionary shards + a separate title inverted index.


In [ ]:

# =========================
# (A) Build doc_id -> title shards
# =========================

from pyspark.sql.functions import col, max as spark_max
import json, pickle
from google.cloud import storage

# --- configure ---
TITLE_COL_CANDIDATES = ["title", "page_title", "doc_title"]
SHARD_SIZE = 100_000
TITLES_SHARDS_PREFIX = "titles_shards"

# detect title column
cols = set(parquetFile.columns)
title_col = None
for c in TITLE_COL_CANDIDATES:
    if c in cols:
        title_col = c
        break

if title_col is None:
    raise ValueError(f"Couldn't find a title column. Available columns: {sorted(cols)[:50]} ...")

print("Using title column:", title_col)


max_id = parquetFile.select(spark_max(col("id")).alias("max_id")).first()["max_id"]
num_shards = int(max_id // SHARD_SIZE) + 1
print("max_id:", max_id, "SHARD_SIZE:", SHARD_SIZE, "num_shards:", num_shards)

def _sid(doc_id: int) -> int:
    return int(doc_id) // SHARD_SIZE


titles_rdd = (
    parquetFile.select(col("id").alias("id"), col(title_col).alias("title"))
    .rdd
    .map(lambda r: (int(r["id"]), "" if r["title"] is None else str(r["title"])))
    .map(lambda x: (_sid(x[0]), (x[0], x[1])))
)


titles_rdd = titles_rdd.partitionBy(num_shards, lambda k: int(k))

def _write_title_shards(partition_iter):
    '''
    Runs on executors.
    Builds {doc_id: title} dict(s) and uploads each shard to:
      gs://<bucket>/{TITLES_SHARDS_PREFIX}/{sid:06d}.pkl
    '''


    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    by_sid = {}
    for sid, (doc_id, title) in partition_iter:
        d = by_sid.setdefault(int(sid), {})
        d[int(doc_id)] = title

    out = []
    for sid, mapping in by_sid.items():
        data = pickle.dumps(mapping, protocol=pickle.HIGHEST_PROTOCOL)
        blob = bucket.blob(f"{TITLES_SHARDS_PREFIX}/{sid:06d}.pkl")
        blob.upload_from_string(data, content_type="application/octet-stream")
        out.append((sid, len(mapping), len(data)))
    return iter(out)

# Trigger writing shards
shard_stats = titles_rdd.mapPartitions(_write_title_shards).collect()
print("Wrote shards (sid, doc_count, bytes):", shard_stats[:5], " ... total:", len(shard_stats))


meta = {
    "shard_size": SHARD_SIZE,
    "num_shards": num_shards,
    "title_col": title_col
}

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
bucket.blob(f"{TITLES_SHARDS_PREFIX}/_meta.json").upload_from_string(
    json.dumps(meta), content_type="application/json"
)

print("Uploaded titles shard meta:", meta)


# Build a Title-only inverted index

Writes postings to `postings_title_gcp/` and index metadata to `postings_title_gcp/title_index.pkl`.


In [ ]:

# =========================
# (B) Build title-only inverted index
# =========================

from pyspark.sql.functions import col
from collections import defaultdict


TITLE_POSTINGS_PREFIX = "postings_title_gcp"
MIN_DF_TITLE = 2   # keep tokens that appear in >= 2 titles (set to 1 if you have enough storage)

# Build (title_text, doc_id) RDD like the body index uses.
doc_title_pairs = parquetFile.select(col(title_col).alias("text"), col("id").alias("id")).rdd

# word_count is already defined earlier in this notebook: def word_count(text, id)
# It lowercases and removes stopwords, no stemming.
title_word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

# posting lists: token -> [(doc_id, tf), ...]
title_postings = title_word_counts.groupByKey().mapValues(reduce_word_counts)

# filter by df threshold
title_postings_filtered = title_postings.filter(lambda x: len(x[1]) >= MIN_DF_TITLE)

# df dict for tokens in title index
title_w2df = calculate_df(title_postings_filtered)
title_w2df_dict = title_w2df.collectAsMap()
print("Title vocab size:", len(title_w2df_dict))

def write_title_bucket(bckt):
    '''
    Writes one bucket's posting lists to:
      gs://<bucket>/postings_title_gcp/<bucket_id>.bin
      gs://<bucket>/postings_title_gcp/<bucket_id>.pickle
    Returns: (bucket_id, token_count, total_postings)
    '''
    bucket_id, tok_pl_iter = bckt
    tok_pl_list = list(tok_pl_iter)
    tok_pl_list.sort(key=lambda x: x[0])

    posting_locs = defaultdict(list)
    TUPLE_SIZE = 6

    # build binary blob
    data = bytearray()
    offset = 0
    total_postings = 0

    bin_name = f"{bucket_id}.bin"
    for token, pl in tok_pl_list:
        posting_locs[token].append((bin_name, offset))
        for doc_id, tf in pl:
            data += int(doc_id).to_bytes(4, "big")
            data += int(tf).to_bytes(2, "big")
        inc = len(pl) * TUPLE_SIZE
        offset += inc
        total_postings += len(pl)

    storage_client = storage.Client()
    b = storage_client.bucket(bucket_name)

    # upload binary postings
    b.blob(f"{TITLE_POSTINGS_PREFIX}/{bin_name}").upload_from_string(
        bytes(data), content_type="application/octet-stream"
    )

    # upload posting_locs for this bucket
    pick_name = f"{bucket_id}.pickle"
    b.blob(f"{TITLE_POSTINGS_PREFIX}/{pick_name}").upload_from_string(
        pickle.dumps(dict(posting_locs), protocol=pickle.HIGHEST_PROTOCOL),
        content_type="application/octet-stream"
    )

    return (bucket_id, len(tok_pl_list), total_postings)

# Partition title postings by token bucket id and write out per bucket
bucket_posting_title = title_postings_filtered.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))
grouped_by_bucket_title = bucket_posting_title.groupByKey()

# Trigger distributed writing
title_write_stats = grouped_by_bucket_title.map(write_title_bucket).collect()
print("Wrote title posting buckets:", title_write_stats[:5], " ... total buckets:", len(title_write_stats))

# Merge posting_locs from bucket pickles (driver-side)
storage_client = storage.Client()
super_posting_locs_title = defaultdict(list)
for blob in storage_client.list_blobs(bucket_name, prefix=TITLE_POSTINGS_PREFIX):
    if not blob.name.endswith(".pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_title[k].extend(v)

print("Merged title posting_locs size:", len(super_posting_locs_title))

# Create and upload the title index metadata pickle (title_index.pkl)
title_index = InvertedIndex()
title_index.posting_locs = super_posting_locs_title
title_index.df = dict(title_w2df_dict)

title_index.write_index(".", "title_index")  # writes title_index.pkl locally

# upload to gs (metadata pickle)
!gsutil cp title_index.pkl gs://{bucket_name}/{TITLE_POSTINGS_PREFIX}/title_index.pkl
print("Uploaded:", f"gs://{bucket_name}/{TITLE_POSTINGS_PREFIX}/title_index.pkl")


In [15]:
def tokenize(text: str):
    if not text:
        return []
    return [m.group() for m in RE_WORD.finditer(text.lower()) if m.group() not in all_stopwords]

In [16]:
ANCHOR_POSTINGS_PREFIX = "postings_anchor_gcp"  # folder inside bucket
MIN_DF_ANCHOR = 2

# RDD: (page_id, anchor_text)
pages_links = parquetFile.select("id", "anchor_text").rdd

def anchor_to_wordcounts(page):
    """
    page[0] = src_id
    page[1] = anchors list: [(dst_id, anchor_string), ...]
    Output: [(token, (dst_id, tf_in_that_anchor_string)), ...]
    """
    anchors = page[1]
    if not anchors:
        return []

    out = []
    for a in anchors:
        if not a or len(a) < 2:
            continue
        dst_id, atext = a[0], a[1]
        tokens = tokenize(atext)
        if not tokens:
            continue
        w2cnt = Counter(tokens)
        did = int(dst_id)
        for w, c in w2cnt.items():
            out.append((w, (did, int(c))))
    return out

def reduce_anchor_posting_list(pl):
    """
    pl is iterable of (doc_id, tf_partial). Need to SUM per doc_id
    because same dst_id can appear many times (many incoming anchors).
    """
    acc = Counter()
    for doc_id, tf in pl:
        acc[int(doc_id)] += int(tf)
    return sorted(acc.items())  # [(doc_id, tf_total), ...]

In [ ]:
# =========================
# (C) Build anchor-text inverted index
# =========================

from collections import defaultdict, Counter


ANCHOR_POSTINGS_PREFIX = "postings_anchor_gcp"
MIN_DF_ANCHOR = 2
TUPLE_SIZE = 6  # 4 bytes doc_id + 2 bytes tf

# --- Build postings: token -> [(doc_id, tf), ...]
pages_links = parquetFile.select("id", "anchor_text").rdd

def anchor_to_wordcounts(page):
    """
    page[0] = src_id
    page[1] = anchors list: [(dst_id, anchor_string), ...]
    Output: [(token, (dst_id, tf_in_that_anchor_string)), ...]
    """
    anchors = page[1]
    if not anchors:
        return []

    out = []
    for a in anchors:
        if not a or len(a) < 2:
            continue
        dst_id, atext = a[0], a[1]
        tokens = tokenize(atext)
        if not tokens:
            continue
        w2cnt = Counter(tokens)
        did = int(dst_id)
        for w, c in w2cnt.items():
            out.append((w, (did, int(c))))
    return out

def reduce_anchor_posting_list(pl):
    """
    pl is iterable of (doc_id, tf_partial). Need to SUM per doc_id
    because same dst_id can appear many times (many incoming anchors).
    """
    acc = Counter()
    for doc_id, tf in pl:
        acc[int(doc_id)] += int(tf)
    return sorted(acc.items())  # [(doc_id, tf_total), ...]

anchor_word_counts = pages_links.flatMap(anchor_to_wordcounts)
anchor_postings = anchor_word_counts.groupByKey().mapValues(reduce_anchor_posting_list)

# Filter by df
anchor_postings_filtered = anchor_postings.filter(lambda x: len(x[1]) >= MIN_DF_ANCHOR)

# DF dict for metadata (can be big!)
anchor_w2df = anchor_postings_filtered.map(lambda x: (x[0], len(x[1])))
anchor_w2df_dict = anchor_w2df.collectAsMap()
print("Anchor vocab size:", len(anchor_w2df_dict))

# ---------- Writer (one .bin + one .pickle per bucket) ----------
def write_anchor_bucket(bckt):
    """
    Writes one bucket's posting lists to:
      gs://<bucket>/<ANCHOR_POSTINGS_PREFIX>/<bucket_id>.bin
      gs://<bucket>/<ANCHOR_POSTINGS_PREFIX>/<bucket_id>.pickle
    Returns: (bucket_id, token_count, total_postings)
    """
    bucket_id, tok_pl_iter = bckt
    tok_pl_list = list(tok_pl_iter)
    tok_pl_list.sort(key=lambda x: x[0])

    posting_locs = defaultdict(list)

    data = bytearray()
    offset = 0
    total_postings = 0

    bin_name = f"{bucket_id}.bin"
    for token, pl in tok_pl_list:
        # store (bin_name, offset) just like your title code
        posting_locs[token].append((bin_name, offset))

        TF_MASK = (1 << 16) - 1
        for doc_id, tf in pl:
            t = (int(doc_id) << 16) | (int(tf) & TF_MASK)
            data += int(t).to_bytes(6, "big")

        inc = len(pl) * TUPLE_SIZE
        offset += inc
        total_postings += len(pl)

    storage_client = storage.Client()
    b = storage_client.bucket(bucket_name)

    # upload binary postings
    b.blob(f"{ANCHOR_POSTINGS_PREFIX}/{bin_name}").upload_from_string(
        bytes(data), content_type="application/octet-stream"
    )

    # upload posting_locs for this bucket
    pick_name = f"{bucket_id}.pickle"
    b.blob(f"{ANCHOR_POSTINGS_PREFIX}/{pick_name}").upload_from_string(
        pickle.dumps(dict(posting_locs), protocol=pickle.HIGHEST_PROTOCOL),
        content_type="application/octet-stream"
    )

    return (bucket_id, len(tok_pl_list), total_postings)

# Partition by bucket id and write
bucket_posting_anchor = anchor_postings_filtered.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))
grouped_by_bucket_anchor = bucket_posting_anchor.groupByKey()

anchor_write_stats = grouped_by_bucket_anchor.map(write_anchor_bucket).collect()
print("Wrote anchor posting buckets:", anchor_write_stats[:5], " ... total buckets:", len(anchor_write_stats))

# ---------- Merge posting_locs from bucket pickles (driver-side) ----------
storage_client = storage.Client()
super_posting_locs_anchor = defaultdict(list)

for blob in storage_client.list_blobs(bucket_name, prefix=ANCHOR_POSTINGS_PREFIX):
    if not blob.name.endswith(".pickle"):
        continue
    posting_locs = pickle.loads(blob.download_as_bytes())   # <-- avoids blob.open() compatibility issues
    for k, v in posting_locs.items():
        super_posting_locs_anchor[k].extend(v)

print("Merged anchor posting_locs size:", len(super_posting_locs_anchor))

# ---------- Create & upload anchor_index.pkl ----------
from inverted_index_gcp import InvertedIndex  # same class you already use

anchor_index = InvertedIndex()
anchor_index.posting_locs = super_posting_locs_anchor
anchor_index.df = dict(anchor_w2df_dict)

# write locally then upload (same style as title)
anchor_index.write_index(".", "anchor_index")  # creates anchor_index.pkl locally
storage_client.bucket(bucket_name).blob(f"{ANCHOR_POSTINGS_PREFIX}/anchor_index.pkl").upload_from_filename("anchor_index.pkl")

print("Uploaded:", f"gs://{bucket_name}/{ANCHOR_POSTINGS_PREFIX}/anchor_index.pkl")

Anchor vocab size: 753533


Wrote anchor posting buckets: [(0, 6083, 1278292), (1, 6087, 1016270), (2, 5950, 917833), (3, 6155, 1115328), (4, 6062, 1460758)]  ... total buckets: 124
Merged anchor posting_locs size: 753533
Uploaded: gs://bucket_323040972/postings_anchor_gcp/anchor_index.pkl


In [ ]:
# =========================
# (D) Build doc_id -> doclen dictionary
# =========================
DOC_LEN_PREFIX = "doclen_shards"
SHARD_SIZE = 100_000

# (text, id) pairs
doc_text_pairs = parquetFile.select("text", "id").rdd

doc_len_rdd = doc_text_pairs.map(lambda x: (int(x[1]), len(tokenize(x[0]))))

# shard id
doc_len_sharded = doc_len_rdd.map(lambda x: (x[0] // SHARD_SIZE, (x[0], x[1])))

def write_kv_shards(prefix, bucket_name, partition_iter):
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    by_sid = {}
    for sid, (doc_id, val) in partition_iter:
        by_sid.setdefault(int(sid), {})[int(doc_id)] = int(val)

    out = []
    for sid, mapping in by_sid.items():
        data = pickle.dumps(mapping, protocol=pickle.HIGHEST_PROTOCOL)
        bucket.blob(f"{prefix}/{sid:06d}.pkl").upload_from_string(
            data, content_type="application/octet-stream"
        )
        out.append((sid, len(mapping), len(data)))
    return iter(out)

stats = doc_len_sharded.mapPartitions(lambda it: write_kv_shards(DOC_LEN_PREFIX, bucket_name, it)).collect()
print("doclen shards:", stats[:3], " ... total:", len(stats))

doclen shards: [(40, 6864, 51394), (41, 14185, 106063), (42, 13532, 101318)]  ... total: 743


In [ ]:
# =========================
# (E) Build doc_id -> pageView dictionary
# =========================
bucket_name = "bucket_323040972"
pv_url = "https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2"


p = Path(pv_url)
pv_name = p.name  # pageviews-202108-user.bz2

# download locally (to the driver node)
#!wget -nc -O {pv_name} {pv_url}

# upload raw file to bucket folder
raw_prefix = "pageviews_raw/2021-08"
client = storage.Client()
bucket = client.bucket(bucket_name)
bucket.blob(f"{raw_prefix}/{pv_name}").upload_from_filename(pv_name)

print(f"Uploaded raw: gs://{bucket_name}/{raw_prefix}/{pv_name}")

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

gs_path = f"gs://{bucket_name}/{raw_prefix}/{pv_name}"
lines = spark.sparkContext.textFile(gs_path)

def parse_monthly_line(line: str):
    # keep English Wikipedia only
    if not line.startswith("en.wikipedia"):
        return None
    parts = line.split(" ")
    if len(parts) < 5:
        return None
    pid, pv = parts[2], parts[4]
    if not (pid.isdigit() and pv.isdigit()):
        return None
    return (int(pid), int(pv))

pv_pairs = lines.map(parse_monthly_line).filter(lambda x: x is not None)

# If duplicates exist, sum them:
pv_sum = pv_pairs.reduceByKey(lambda a, b: a + b)  # (page_id, month_views)


import pickle

PAGEVIEWS_PREFIX = "pageviews_shards/2021-08"
SHARD_SIZE = 100_000

def write_kv_shards(prefix, bucket_name, partition_iter):
    
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    by_sid = {}
    for sid, (doc_id, val) in partition_iter:
        by_sid.setdefault(int(sid), {})[int(doc_id)] = int(val)

    out = []
    for sid, mapping in by_sid.items():
        data = pickle.dumps(mapping, protocol=pickle.HIGHEST_PROTOCOL)
        bucket.blob(f"{prefix}/{sid:06d}.pkl").upload_from_string(
            data, content_type="application/octet-stream"
        )
        out.append((sid, len(mapping)))
    return iter(out)

pv_sharded = pv_sum.map(lambda x: (x[0] // SHARD_SIZE, (x[0], x[1])))
stats = pv_sharded.mapPartitions(lambda it: write_kv_shards(PAGEVIEWS_PREFIX, bucket_name, it)).collect()

print("Wrote pageview shards. Example:", f"gs://{bucket_name}/{PAGEVIEWS_PREFIX}/000000.pkl")
print("Shard stats sample:", stats[:5])


--2026-01-12 13:57:56--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  3.88MB/s    in 10m 0s  

2026-01-12 14:07:57 (3.98 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]

Uploaded raw: gs://bucket_323040972/pageviews_raw/2021-08/pageviews-202108-user.bz2


26/01/12 14:08:16 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


Wrote pageview shards. Example: gs://bucket_323040972/pageviews_shards/2021-08/000000.pkl
Shard stats sample: [(12, 2286), (64, 966), (29, 1226), (49, 1039), (52, 983)]
